In [1]:
import numpy as np
import opensimplex
import random
from time import sleep
import scipy.linalg as sla
import numba 

In [2]:
@numba.jit(nogil = True)
def normal_random_distribution(data_array, X_array, expd_vl):
    print("here")
    EPSILON = 0.001
    data_len = data_array.size
    data_array[:] = 1 / data_len
    current_E = np.sum(data_array * X_array)
    middle = np.argmax(X_array > expd_vl) - 1
    while abs(current_E - expd_vl) > EPSILON:
        left_i = random.randint( 0, middle )
        rght_i = random.randint( middle + 1, data_len - 1 )
        if current_E < expd_vl:
            if data_array[left_i] < EPSILON:
                continue
            up_lim = min(data_array[left_i] , ( expd_vl - current_E ) / ( X[rght_i] - X[left_i] ) )
            mu = up_lim / 2
            sigma = up_lim / 6
            x = np.clip(np.random.normal( mu, sigma, 1 ), 0, up_lim)[0]
            data_array[left_i] -= X
            data_array[rght_i] += x
            current_E += x * ( X[rght_i] - X[left_i] )
        else:
            if data_array[rght_i] < EPSILON:
                continue
            up_lim = min(data_array[rght_i] , ( current_E - expd_vl ) / ( X[rght_i] - X[left_i] ) ) 
            mu = up_lim / 2
            sigma = up_lim / 6
            x = np.clip(np.random.normal( mu, sigma, 1 ), 0, up_lim)[0]
            data_array[left_i] += X
            data_array[rght_i] -= x
            current_E -= x * ( X[rght_i] - X[left_i] )

In [4]:
#     1)number of stores 
#     2)number of products :- an int representing the number of products
#     3timeframe :- an int representing the number of days we simulate
#     4)factory_capacity :- an int representing how many total products the factory can produce in a day
#     5)DC_capacity :- an int representing how many total products can be stored:
#     6)demand range min :- an int saying the least value of each products order
#     7)demand range max :- an int saying the max value of each products order
#     8)truck_capacity :- total products truck can carry
#     9)data = array of shape (stores, products, demand_range, timeframe) where data[w][x][y][z] = probability of
#         store w wanting product x in (y + demand_range_min) amount on day z 
#     10)truck_max_cap :- hypothetically, the number of trucks on a day can be min(max_supply, max_demand)
#         i.e. 1 truck per product from demand to supply 
#         max_supply = DC_capacity
#         max_demand = demand_range_max * number of stores * number of products
#         but simulating this wastes a lot of space, so we resonably cap the number of trucks to save space
#     11)DC_cap_to_noise_mean_factor: -I generate 4d simplex noise and multiply that with DC_Capacity to get 
#             data array, but if demand always tends towards max supply then because of inoptimality 
#         it will keep on cummilating, keeping this near 1 is like a stress test for the algorithm

class TestCase:
    def __init__(self, number_of_stores,number_of_products, timeframe = 90, factory_capacity = 200, 
                       DC_capacity = 50, demand_range_min = 10, demand_range_max = 400,
                      truck_capacity = 20, truck_max_cap = 0.1):
        self.number_of_stores = number_of_stores
        self.number_of_products = number_of_products
        self.timeframe = timeframe
        self.factory_capacity = factory_capacity
        self.DC_capacity = DC_capacity
        self.demand_range_min = demand_range_min
        self.demand_range_max = demand_range_max
        self.truck_capacity = truck_capacity
        self.truck_max_cap = truck_max_cap

        self.maximum_truck = truck_max_cap * min(DC_capacity, demand_range_max * number_of_stores*number_of_products )
    
    def getData(self, DC_cap_to_noise_mean_factor = 1, seed = None, simplex_reduction_factor = 0.3,
                expected_value_to_distribution = "normal random distribution" ):
#         simplex_reduction_factor :- making this value low will reduce variance and all data will look the same
#                                    keeping it high will make the data look completely random 
        demand_range = self.demand_range_max - self.demand_range_min + 1
        data = np.zeros((self.number_of_stores, self.number_of_products, self.timeframe, demand_range), 
                        dtype=np.float32)
        if seed == None:
            opensimplex.random_seed()
        else:
            opensimplex.seed(seed)
        for store in np.arange(self.number_of_stores):
            for product in np.arange(self.number_of_products):
                for time in np.arange(self.timeframe):
                    data[store, product, time, 0] = opensimplex.noise3(store*simplex_reduction_factor, 
                                                                       product*simplex_reduction_factor, 
                                                                       time*simplex_reduction_factor)
        #normalising from -1,1 -> 0, 1
        data[:, :, :, 0] += 1
        data[:, :, :, 0] /= 2
        
        total = np.sum(data[:,:,:,0])
        cumm_dmnd_ = self.timeframe * self.DC_capacity * DC_cap_to_noise_mean_factor
        data[:, :, :, 0] *= cumm_dmnd_ / total 

                
                
        if not callable(expected_value_to_distribution):
            if expected_value_to_distribution == "normal random distribution":
                expected_value_to_distribution = normal_random_distribution
        
        for store in np.arange(self.number_of_stores):
            for product in np.arange(self.number_of_products):
                for time in np.arange(self.timeframe):
                    expd_vl = data[store, product, time, 0]
                    expected_value_to_distribution(data[store, product, time], 
                                                   np.arange(self.demand_range_min, self.demand_range_max + 1),
                                                   expd_vl)
                    return data
#                     print(np.sum(data[store, product, time]))
        return data
tc = TestCase(2,2, DC_capacity=200)
hlpr = tc.getData(DC_cap_to_noise_mean_factor = 1, simplex_reduction_factor = 0.3)

/tmp/ipykernel_322463/3112710218.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "normal_random_distribution" failed type inference due to: NameError: name 'X' is not defined
  @numba.jit(nogil = True)


ValueError: Failed in object mode pipeline (step: object mode frontend)
not enough values to unpack (expected 1, got 0)

In [ ]:
tc = TestCase(2,2, DC_capacity=200)
hlpr = tc.getData(DC_cap_to_noise_mean_factor = 1, simplex_reduction_factor = 0.3)
# print(np.sum(hlpr < 25)*100 / hlpr.size) 
# hlpr

In [ ]:
np.arange(10,20, dtype=np.int32)
random.uniform(20,10)
# 5//2
np.array([1,2,3,4]) * np.array([1,2,3,4])
E=np.array(
    [
        [-2,-4,1,-9,0],
        [3,6,0,12,3],
        [1,2,1,3,1],
        [-5,-10,3,-23,1]
    ]
)
sla.null_space(E)
# E.shape

In [ ]:
np.random.normal(60, 10, 1)[0]